In [77]:
import pandas as pd

file_path = '/Users/dariagerenkova/Desktop/задание на аналитика.xlsx'
r1 = pd.read_excel(file_path, sheet_name='отчет 1')
r2 = pd.read_excel(file_path, sheet_name='отчет 2')
cession = pd.read_excel(file_path, sheet_name='цессия')

merged = r1.merge(
    r2,
    left_on='Номер контракта',
    right_on='NumContract',
    how='left'
)

excluded_status = ['REPAID', 'EARLY_REPAID', 'BANKRUPTCY']
cleaned = merged[
    ~merged['Номер контракта'].isin(cession['ContractID']) &
    ~merged['Status'].isin(excluded_status)
]

pivot = cleaned.pivot_table(
    index='SubdivisionName',
    values=['Номер контракта', 'LoanAmount'],
    aggfunc={'Номер контракта': 'count', 'LoanAmount': 'sum'}
).rename_axis('Регион').reset_index().rename(columns={
    'Номер контракта': 'Кол-во договоров',
    'LoanAmount': 'Сумма выдач'
})

print('Сводная таблица по регионам')
display(pivot)
pivot.to_excel('by_region.xlsx', index=False)

Сводная таблица по регионам


,Регион,Сумма выдач,Кол-во договоров
0,Астрахань,30000,1
1,Белгород,65000,2
2,Вологда,30000,1
3,Курск,20000,1
4,Онлайн,127400,13
5,Пенза,30000,1
6,Санкт-Петербург,50000,1
7,Электросталь,100000,2


In [81]:
cleaned = cleaned.drop(columns=['NumContract', 'ClientID', 'inner_lead_id', 'issueDateTimestamp', 'RepaymentPlanDate', 'LoanAmount'])
cleaned = cleaned.rename(columns={
    'SubdivisionName': 'Регион выдачи',
    'Status': 'Статус',
    'DateStatus': 'Дата последнего платежа',
    'SumLastPay': 'Сумма последнего платежа',
    'ContractID': 'ID договора' 
})
print('Исходный реестр')
display(cleaned)

Исходный реестр


,№,Номер контракта,дата выдачи,Сумма выдачи,Задолженность по ОД,Задолженность по %%,"Кол-во дней просрочки, фактическое",ID договора,Регион выдачи,Статус,Дата последнего платежа,Сумма последнего платежа
0,1,3547,2023-03-23,60000,47104.16,23926.86,525,10614,Электросталь,COLLECTOR,2023-11-21 23:00:00,1775
1,2,68,2022-09-30,35000,11447.66,1300.06,552,183,Белгород,COLLECTOR,2024-10-03 14:33:23,1950
2,3,389,2022-10-28,30000,4410.20,249.72,482,1146,Вологда,DELINQUENT,2023-10-07 00:00:00,1200
3,4,1393,2022-12-20,40000,11088.24,1482.47,457,4157,Электросталь,ACTIVE_SB,2023-12-12 23:00:00,3000
4,5,2050,2023-01-26,50000,42219.24,30483.14,609,6127,Санкт-Петербург,COLLECTOR,2024-12-17 15:34:37,2200
5,6,2154,2023-01-31,20000,6606.81,864.92,583,6439,Курск,COLLECTOR,2024-12-17 15:34:38,6652.05
6,7,2148,2023-01-31,30000,8796.44,1169.45,415,6421,Пенза,ACTIVE_SB,2024-01-23 23:00:00,9000
7,8,3211,2023-03-10,30000,25533.57,19657.32,573,9610,Белгород,COLLECTOR,2023-09-11 09:15:44,4250
8,9,3512,2023-03-21,30000,26037.23,21091.66,569,10509,Астрахань,COLLECTOR,2023-09-13 08:46:09,1100
9,10,6081,2023-06-18,9500,9500.00,5157.97,578,18155,Онлайн,COLLECTOR,2024-12-17 15:34:43,\N
